In [1]:
from pandas import Series,DataFrame
from numpy import *
import numpy as np
import pandas as pd
import pandas
import codecs
import torch 
import torch.nn as nn 
from torch.autograd import Variable 
import torch.utils.data as Data 
import torchvision 
import matplotlib.pyplot as plt 
import csv
from sklearn.model_selection import cross_val_score,train_test_split

In [2]:
df = pd.read_excel(r'sel_all.xlsx',header =0)

In [3]:
feature = df[['LipoaffinityIndex','MLogP','maxHsOH','minsssN','MDEC-23','nC','maxssO',
             'minHsOH','BCUTc-1l','maxsssN','MLFER_A','minssO','SHsOH',
              'SsOH','maxsOH','ATSp5','SwHBa','C1SP2','nHBAcc','minHBint5']]
#feature = df[['C1SP2','nHBAcc','ATSc2','MDEC-23','C3SP2','nHBAcc_Lipinski','ATSc1',
#              'ETA_BetaP_s','BCUTc-1h']]



In [4]:
pIC50 = df.values[:, -1]
pIC50 = pd.DataFrame(pIC50)

In [7]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(20, 1000),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(1000, 800),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(800, 600),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(600, 500),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(500, 300),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(300, 250),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(250, 1)
        )

        # init

    def forward(self, x):
        x = self.mlp(x)
        return x
#model_dict=torch.load("ANN.pth")

In [7]:
feature = feature.astype(float)
feature = torch.from_numpy(feature.values)
feature = feature.float()
pIC50 = pIC50.astype(float)
pIC50 = torch.from_numpy(pIC50.values)
pIC50 = pIC50.float()


In [9]:
lossData=[[]]
model = ANN()

In [15]:
loss_func = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), 0.0001)
batch_size =64
epochs = 200
print("epoch\t loss\t")
losses = []
for i in range(epochs):
     batch_loss = []
    # start和end分别是提取一个batch数据的起始和终止下标
     for start in range(0, len(feature), batch_size):
        end = start + batch_size if start + batch_size < len(feature) else len(feature)
        xx = torch.tensor(feature[start:end], dtype = torch.float, requires_grad = False)
        yy = torch.tensor(pIC50[start:end], dtype = torch.float, requires_grad = False)
        predict = model(xx)
        loss =loss_func(predict, yy)
        optim.zero_grad()
        loss.backward()
        optim.step()
        batch_loss.append(loss.data.numpy())
    
    # 每隔100步输出一下损失值（loss）
     if i % 10==0:
        losses.append(np.mean(batch_loss))
        print(i, np.mean(batch_loss))

epoch	 loss	
0

<ipython-input-15-db6a3aa0ddb1>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xx = torch.tensor(feature[start:end], dtype = torch.float, requires_grad = False)
<ipython-input-15-db6a3aa0ddb1>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yy = torch.tensor(pIC50[start:end], dtype = torch.float, requires_grad = False)


 1.5836023
10 1.1053729
20 1.1023707
30 1.1978526
40 1.094581
50 1.1963027
60 1.2005458
70 1.1519866
80 1.1269196
90 1.1101768
100 1.0952128
110 1.2230247
120 1.0663292
130 1.0883863
140 1.2265183
150 1.0828196
160 1.1023023
170 1.102582
180 1.081222
190 1.0940244


In [17]:
torch.save(model,"ANN1.pth")

In [8]:
import pyswarms
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
from pyswarms.single.global_best import GlobalBestPSO
from pyswarm import pso

In [9]:
model_dict_ANN=torch.load("ANN1.pth")

In [10]:
def Func(a):
    a= torch.tensor(a)
    a = a.to(torch.float32)
    j = -model_dict_ANN(a)
    k = j.detach().numpy()
    return k


In [38]:
a = Func(feature.values[1,])

In [39]:
a

array([-6.1592464], dtype=float32)

In [37]:
feature.values[1,]

array([ 5.79216498e+00,  3.11000000e+00,  4.99770093e-01,  0.00000000e+00,
        2.29985968e+01,  1.80000000e+01,  0.00000000e+00,  2.57051918e-01,
       -3.90588644e-01,  0.00000000e+00,  1.14700000e+00,  0.00000000e+00,
        1.07109615e+00,  3.02024609e+01,  1.03910051e+01,  2.66847641e+03,
        8.84471444e+00,  0.00000000e+00,  2.00000000e+00,  0.00000000e+00])

In [12]:
variable_range=pd.read_excel(r'第四题取值范围.xlsx').values#  可变参数的取值范围
lb = variable_range[0,1:]
ub = variable_range[1,1:]
lb=np.array(lb,dtype=np.float)
ub=np.array(ub,dtype=np.float)


In [44]:
#options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9} # 粒子群算法的参数
#optimizer = GlobalBestPSO(n_particles=20, dimensions=20, options=options, bounds=bounds)
#cost, pos = optimizer.optimize(Func, iters=1000)

In [53]:
xopt, fopt = pso(Func, lb, ub, f_ieqcons=None, args=(), kwargs={},
    swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=120, minstep=1e-8,
    minfunc=1e-8, debug=False)


Stopping search: maximum iterations reached --> 120


In [54]:
print(xopt , fopt)

[ 6.61088022e+00  3.13483547e+00  8.45795817e-01  2.90563041e-02
  3.63222191e+01  2.29470590e+01  1.55842584e+00  1.35166661e-01
 -2.88584434e-01  1.53167211e+00  1.08038021e+00  6.18007706e-01
  2.30553964e+00  3.97200000e+01  3.24311274e+00  5.25000000e+02
  4.05276790e+01  4.85985999e+00  8.12711228e-02  1.04593882e+01] [-12.203024]
